# FrankenBert demonstration notebook

In [55]:
!pip install transformers
!pip install datasets

In [56]:
!git clone https://github.com/hopl1t/frankenbert.git

Cloning into 'frankenbert'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 122 (delta 58), reused 74 (delta 25), pack-reused 0
Receiving objects: 100% (122/122), 275.37 KiB | 3.58 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [57]:
import os
os.chdir('frankenbert')
!git checkout nir
from trainer import *
import utils
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

Branch 'nir' set up to track remote branch 'nir' from 'origin'.
Switched to a new branch 'nir'


In [58]:
CAUSAL_DIR = '/content/drive/MyDrive/NLP_project/trained_models/causal'
MLM_DIR = '/content/drive/MyDrive/NLP_project/trained_models/mlm'
CSV_DIR = '/content/drive/MyDrive/NLP_project/csvs'

## Causal models

In [59]:
gpt_tokenizer = AutoTokenizer.from_pretrained('distilgpt2', use_fast=True)
wiki_causal_model = torch.load(os.path.join(CAUSAL_DIR, 'distilgpt2_wikitext_2109071909.pkl'))
shakespeare_causal_model = torch.load(os.path.join(CAUSAL_DIR, 'distilgpt2_tiny_shakespeare_2109072009.pkl'))
web_causal_model = torch.load('/content/drive/MyDrive/NLP_project/trained_models/causal/distilgpt2_stas/openwebtext-10k_2109082209.pkl')

#### Predict next word

In [ ]:
sentences = ['Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would',
             'When you get right down to the bottom of the barrel',
             'The crow scavenged for the loose corn kernels flying through the ionic sea of',
             'I love this app it is sooo much fun and it is sooo',
             'How low can you go after the water stops rising and the world is burning but']
norm_columns = ['web', 'wiki', 'shakespeare']
swap_columns = [[[i] for i in range(6)],
                [[i, i+1] for i in range(5)],
                [[i, i+1, i+2] for i in range(4)],
                [[i, i+1, i+2, i+3] for i in range(3)],
                [[0,1,2,3,4,5]]]
cols = []
for columns in swap_columns:
    for layer in columns:
        cols.append(layer.__repr__())
web_wiki_df = pd.DataFrame(index=sentences, columns=cols)
shakespeare_wiki_df = pd.DataFrame(index=sentences, columns=cols)
web_shakespeare_df = pd.DataFrame(index=sentences, columns=cols)
wiki_shakespeare_df = pd.DataFrame(index=sentences, columns=cols)

norm_df = pd.DataFrame(index=sentences, columns=norm_columns)
for sentence in sentences:
    norm_df.loc[sentence]['web'] = utils.predict_next(web_causal_model, sentence, gpt_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
    norm_df.loc[sentence]['wiki'] = utils.predict_next(wiki_causal_model, sentence, gpt_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
    norm_df.loc[sentence]['shakespeare'] = utils.predict_next(shakespeare_causal_model, sentence, gpt_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
norm_df.to_csv(os.path.join(CSV_DIR, 'completion', 'norm.csv'))

for layers in swap_columns:
    for layer in layers:
        layer = layer.__repr__()        
        web_wiki_model = utils.frankenstein(web_causal_model, wiki_causal_model, layer)
        web_shakespeare_model = utils.frankenstein(web_causal_model, shakespeare_causal_model, layer)
        shakespeare_wiki_model = utils.frankenstein(shakespeare_causal_model, shakespeare_causal_model, layer)
        wiki_shakespeare_model = utils.frankenstein(shakespeare_causal_model, shakespeare_causal_model, layer)
        for sentence in sentences:
            web_wiki_df.loc[sentence][layer] = utils.predict_next(web_wiki_model, sentence, gpt_tokenizer)
            web_shakespeare_df.loc[sentence][layer] = utils.predict_next(web_shakespeare_model, sentence, gpt_tokenizer)
            shakespeare_wiki_df.loc[sentence][layer] = utils.predict_next(shakespeare_wiki_model, sentence, gpt_tokenizer)
            wiki_shakespeare_df.loc[sentence][layer] = utils.generate(wiki_shakespeare_model, sentence, gpt_tokenizer)
web_wiki_df.to_csv(os.path.join(CSV_DIR, 'completion', 'web_wiki.csv'))
web_shakespeare_df.to_csv(os.path.join(CSV_DIR, 'completion', 'web_shakespeare.csv'))
shakespeare_wiki_df.to_csv(os.path.join(CSV_DIR, 'completion', 'shakespeare_wiki.csv'))

In [43]:
print('===== norm df =====')
display(norm_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== web-wiki df =====')
display(web_wiki_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== web-shakespeare df =====')
display(web_shakespeare_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== shakespeare-wiki df =====')
display(shakespeare_wiki_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== wiki-shakespeare df =====')
display(wiki_shakespeare_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))

===== norm df =====


,web,wiki,shakespeare
Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would,0.723: be 0.0573: make 0.036: work,0.739: be 0.0502: make 0.0341: have,0.663: be 0.0436: not 0.0402: \n
When you get right down to the bottom of the barrel,"0.624: , 0.102: and 0.053: of","0.479: , 0.169: and 0.0377: of","0.55: , 0.084: \n 0.0676: :"
The crow scavenged for the loose corn kernels flying through the ionic sea of,0.588: the 0.0567: space 0.0289: water,0.46: the 0.0405: Africa 0.0404: Jupiter,0.241: the 0.0803: heaven 0.0657: France
I love this app it is sooo much fun and it is sooo,0.186: much 0.157: fun 0.0989: awesome,0.214: much 0.144: fun 0.108: good,0.45: much 0.103: good 0.0454: \n
How low can you go after the water stops rising and the world is burning but,0.187: it 0.172: the 0.137: you,0.224: the 0.145: it 0.0979: you,0.168: \n 0.123: not 0.116: the


===== web-wiki df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would,0.707: be 0.0549: make 0.0384: work,0.735: be 0.0549: make 0.0309: have,0.713: be 0.0656: make 0.0339: work,0.741: be 0.0493: make 0.0338: have,0.721: be 0.0595: make 0.0395: work,0.726: be 0.0481: make 0.034: work,0.722: be 0.0506: make 0.0363: have,0.728: be 0.0623: make 0.029: have,0.734: be 0.0566: make 0.0313: have,0.741: be 0.0508: make 0.0326: have,0.724: be 0.0509: make 0.0372: work,0.715: be 0.0567: make 0.0344: have,0.745: be 0.055: make 0.0293: have,0.734: be 0.0581: make 0.03: have,0.742: be 0.0441: make 0.0329: have,0.729: be 0.051: make 0.0352: have,0.746: be 0.0564: make 0.0283: have,0.735: be 0.0507: make 0.0301: have,0.726: be 0.0463: make 0.0355: have
When you get right down to the bottom of the barrel,"0.617: , 0.102: and 0.0497: of","0.592: , 0.103: and 0.0565: of","0.618: , 0.109: and 0.042: of","0.607: , 0.116: and 0.0419: you","0.591: , 0.112: and 0.0551: of","0.439: , 0.15: and 0.0748: of","0.583: , 0.105: and 0.0518: of","0.581: , 0.112: and 0.0434: of","0.596: , 0.123: and 0.0447: you","0.555: , 0.129: and 0.0462: you","0.384: , 0.158: and 0.077: of","0.563: , 0.115: and 0.0473: you","0.552: , 0.126: and 0.0512: you","0.527: , 0.14: and 0.0498: you","0.326: , 0.173: and 0.0792: you","0.525: , 0.129: and 0.0592: you","0.478: , 0.146: and 0.0567: you","0.289: , 0.182: and 0.0837: you","0.228: , 0.184: and 0.108: you"
The crow scavenged for the loose corn kernels flying through the ionic sea of,0.509: the 0.0344: water 0.0294: space,0.61: the 0.0405: space 0.0296: a,0.6: the 0.0544: space 0.0291: a,0.584: the 0.0522: space 0.0286: water,0.597: the 0.0438: space 0.0308: water,0.587: the 0.0546: space 0.0362: water,0.5: the 0.0284: New 0.022: a,0.615: the 0.0357: space 0.0294: a,0.581: the 0.0509: space 0.0259: a,0.592: the 0.0377: space 0.0298: water,0.591: the 0.0444: space 0.0386: water,0.497: the 0.0294: New 0.0224: Florida,0.584: the 0.034: space 0.0254: a,0.589: the 0.0366: space 0.0242: water,0.587: the 0.0402: space 0.0375: water,0.458: the 0.036: New 0.0242: Florida,0.587: the 0.0246: space 0.0234: a,0.594: the 0.0387: space 0.0302: water,0.502: the 0.0271: Africa 0.0264: New
I love this app it is sooo much fun and it is sooo,0.198: much 0.141: fun 0.113: awesome,0.178: fun 0.173: much 0.0988: awesome,0.165: fun 0.158: much 0.102: awesome,0.191: much 0.154: fun 0.093: easy,0.184: much 0.174: fun 0.0944: awesome,0.196: much 0.165: fun 0.104: awesome,0.173: much 0.16: fun 0.108: awesome,0.182: fun 0.15: much 0.0996: good,0.165: much 0.16: fun 0.0958: good,0.189: much 0.173: fun 0.0904: easy,0.198: much 0.18: fun 0.0986: awesome,0.154: fun 0.151: much 0.112: good,0.175: fun 0.161: much 0.109: good,0.183: fun 0.164: much 0.102: good,0.205: much 0.179: fun 0.103: good,0.156: much 0.149: fun 0.126: good,0.199: fun 0.164: much 0.115: good,0.185: fun 0.178: much 0.115: good,0.166: much 0.162: fun 0.155: good
How low can you go after the water stops rising and the world is burning but,0.193: it 0.18: the 0.138: you,0.191: it 0.163: the 0.136: you,0.178: it 0.173: the 0.122: you,0.178: it 0.177: the 0.143: you,0.182: the 0.173: it 0.129: you,0.181: you 0.18: it 0.17: the,0.196: it 0.169: the 0.136: you,0.181: it 0.164: the 0.12: you,0.18: the 0.169: it 0.128: you,0.187: the 0.163: it 0.136: you,0.18: you 0.177: the 0.162: it,0.186: it 0.17: the 0.119: you,0.172: it 0.169: the 0.126: you,0.19: the 0.155: it 0.12: you,0.19: you 0.181: the 0.151: it,0.178: it 0.175: the 0.125: you,0.178: the 0.156: it 0.12: you,0.187: the 0.169: you 0.144: it,0.18: the 0.174: you 0.151: it


===== web-shakespeare df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would,0.712: be 0.0535: make 0.0396: work,0.762: be 0.0471: make 0.0299: have,0.728: be 0.0526: make 0.034: have,0.713: be 0.0515: make 0.0381: work,0.697: be 0.0592: make 0.0433: work,0.715: be 0.0495: make 0.0367: work,0.751: be 0.04: make 0.0325: have,0.765: be 0.0423: make 0.0315: have,0.718: be 0.0462: make 0.0389: have,0.687: be 0.052: make 0.0458: work,0.685: be 0.0516: make 0.0438: work,0.741: be 0.0364: make 0.0357: have,0.754: be 0.0364: have 0.0361: make,0.694: be 0.046: make 0.0401: have,0.674: be 0.0456: make 0.0453: work,0.722: be 0.0423: have 0.0407: not,0.729: be 0.0373: have 0.0362: make,0.682: be 0.0402: make 0.039: have,0.663: be 0.0451: not 0.0397: have
When you get right down to the bottom of the barrel,"0.604: , 0.0927: and 0.0599: of","0.592: , 0.0952: and 0.0558: of","0.638: , 0.094: and 0.0523: of","0.622: , 0.0969: and 0.0568: of","0.595: , 0.107: and 0.0552: of","0.638: , 0.0986: and 0.0502: of","0.563: , 0.0836: and 0.0674: of","0.592: , 0.082: and 0.059: of","0.633: , 0.0861: and 0.0563: of","0.591: , 0.1: and 0.0576: of","0.621: , 0.101: and 0.0507: of","0.561: , 0.0726: of 0.0671: and","0.576: , 0.0717: and 0.0619: of","0.597: , 0.0877: and 0.0552: of","0.624: , 0.0935: and 0.0511: of","0.548: , 0.0715: of 0.0639: .","0.525: , 0.0682: and 0.0622: .","0.636: , 0.0799: and 0.0465: of","0.504: , 0.08: ? 0.0791: ."
The crow scavenged for the loose corn kernels flying through the ionic sea of,0.556: the 0.0321: water 0.0301: night,0.583: the 0.0457: space 0.0313: a,0.555: the 0.0518: space 0.0304: a,0.523: the 0.0546: space 0.0302: water,0.522: the 0.0428: space 0.0284: water,0.508: the 0.0634: space 0.0456: water,0.586: the 0.0293: his 0.0267: heaven,0.531: the 0.0394: space 0.0333: a,0.479: the 0.0472: space 0.0277: Saturn,0.441: the 0.0362: space 0.0277: water,0.438: the 0.0459: space 0.0429: water,0.501: the 0.0453: heaven 0.0378: his,0.439: the 0.0328: space 0.032: Venus,0.395: the 0.0426: Venus 0.0368: Jupiter,0.363: the 0.0444: Venus 0.0407: water,0.412: the 0.0596: heaven 0.039: his,0.34: the 0.0503: Venus 0.0432: Jupiter,0.314: the 0.0721: Venus 0.0496: Jupiter,0.238: the 0.0736: Venus 0.0506: Jupiter
I love this app it is sooo much fun and it is sooo,0.242: much 0.14: good 0.115: fun,0.192: fun 0.181: much 0.0949: good,0.181: fun 0.155: much 0.105: awesome,0.274: much 0.142: fun 0.0798: awesome,0.222: much 0.182: fun 0.089: easy,0.227: much 0.173: fun 0.0766: good,0.285: much 0.161: good 0.103: fun,0.215: fun 0.158: much 0.107: good,0.226: much 0.167: fun 0.0899: good,0.323: much 0.163: fun 0.0724: easy,0.268: much 0.197: fun 0.0665: great,0.242: much 0.175: good 0.108: fun,0.238: much 0.186: fun 0.111: good,0.27: much 0.201: fun 0.0744: good,0.372: much 0.175: fun 0.0628: good,0.342: much 0.147: good 0.0794: fun,0.284: much 0.222: fun 0.0879: good,0.314: much 0.219: fun 0.0683: good,0.423: much 0.105: fun 0.0901: good
How low can you go after the water stops rising and the world is burning but,0.249: the 0.15: you 0.146: it,0.193: the 0.177: it 0.132: you,0.172: the 0.158: it 0.122: you,0.173: it 0.165: the 0.146: you,0.166: it 0.158: the 0.143: you,0.182: it 0.163: you 0.145: the,0.269: the 0.135: you 0.131: it,0.183: the 0.134: it 0.111: you,0.162: the 0.142: it 0.126: you,0.151: you 0.15: it 0.15: the,0.168: you 0.158: it 0.13: the,0.242: the 0.107: you 0.0915: not,0.166: the 0.125: not 0.115: it,0.147: the 0.127: you 0.125: it,0.175: you 0.143: it 0.122: the,0.211: the 0.104: not 0.0981: you,0.148: the 0.14: not 0.109: you,0.142: you 0.12: it 0.118: the,0.132: the 0.11: not 0.0927: you


===== shakespeare-wiki df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:,0.663: be 0.0436: not 0.0402:
When you get right down to the bottom of the barrel,"0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :","0.55: , 0.084: 0.0676: :"
The crow scavenged for the loose corn kernels flying through the ionic sea of,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France,0.241: the 0.0803: heaven 0.0657: France
I love this app it is sooo much fun and it is sooo,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:,0.45: much 0.103: good 0.0454:
How low can you go after the water stops rising and the world is burning but,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the,0.168: 0.123: not 0.116: the


===== wiki-shakespeare df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would,"Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would have been better, more or less to kill. And with 'tis a good day! WARWICK: Go to Buckingham; he says, I pray he is a traitor with the queen's crown: but I hear the queen's worsen of her sister's throne is dead: but as his name will be changed, let me bid farewell to her. KING RICHARD III: There is a good occasion when I wish a person to be pardoned, of the king's crown, and the queen for having made a great state of peace, as in that his daughter, Isabel, had a noble daughter. KING RICHARD III: I know, at least to do so, of my own. I was so near to him, at least not yet. GLOUCESTER: It is a pity, as a word'st that she is not wounded by death, but, in that you would give up some tit to her: when she was to be given a man's crown, she would go,","Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would help. ISABELLA: That is, they told me that 'bought fruit; and it did, To give you the good-run'd root. MERCUTIO: There is no word as to why you should be so mad! ISABELLA: Thou art more a matter of revenge than revenge. BENVOLIO: Ay, what? MERCUTIO: No, what am I supposed to say? What's not? MERCUTIO: I am certain, but it is quite true and wrong. PERCUTIO: Well, no, in answer to your request, but I may at least, To give you a certain answer: First, be not more mad; I will not, I do be too mad, when it comes to your request. My Lord, is there in time which I would prefer for you That you will help me? I shall do so, That, as for you, I myself desire you to help. DUKE OF","Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would be much worse than this? BRUTUS: As far as I know, it is too late. MENENIUS: But that can be undone. I'll kill you, That you are not happy with me. First Citizen: I have a husband and two children. MENENIUS: If, after having heard your thoughts, In any degree I will do you wrong. BRUTUS: As much as I desire to kill you that I do, And I'll kill you that I would. First Citizen: Ah, that I have heard your thought, In any kind of mind, in a way. First Citizen: Or what? MENENIUS: The last in a very short time, The last of the ages. First Citizen: You, that's how that was, The last of the ages; I had the most gentle and loving wife, I did nothing wrong with you; But, in any kind of mind, I was happy. I","Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would be a fool to make it out of one piece, like a madman! Well I had this. KING HENRY VI: What, my lord, was that? DUKE VINCENTIO: Hoping you all may not be quite so to speak But do you have so much information, let's make some friends for him: You need a list of friends, a very good, and, of course, you should be at least some friends for him. DUKE VINCENTIO: Here, sir, is he? DUKE VINCENTIO: Well, sir, what's his name? DUKE VINCENTIO: I do not, sir, I love him; I love him, well I love him, well I love him, well I love him, well And so, sir, and so. DUKE VINCENTIO: Well, sir, what, good friends, good friends? DUKE VINCENTIO: I am more","Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would be better served DUCHESS OF YORK: Well, if so, would you Like to eat it with 'em, and not with 'em For the common purpose of giving it more. ' LADY ANNE: You see it is, that he hath seen With all the sudden in a time; To do this, I suppose, is not enough, by any fault To prove such a thing. DUCHESS OF YORK: Well, if so, would you like to hear it more Away then, would you like to hear it more? LADY ANNE: So, if so, is not enough; For more, I suppose, is it too, you are but not enough To know and let it play. DUCHESS OF YORK: No, sir,

#### Text generation

In [ ]:
sentences = ['A',
             'As of',
             'Indeed,',
             'Nonetheless,',
             'In the year',
             'Following the recommendation']
norm_columns = ['web', 'wiki', 'shakespeare']
swap_columns = [[[i] for i in range(6)],
                [[i, i+1] for i in range(5)],
                [[i, i+1, i+2] for i in range(4)],
                [[i, i+1, i+2, i+3] for i in range(3)],
                [[0,1,2,3,4,5]]]
cols = []
for columns in swap_columns:
    for layer in columns:
        cols.append(layer.__repr__())
web_wiki_df = pd.DataFrame(index=sentences, columns=cols)
web_shakespeare_df = pd.DataFrame(index=sentences, columns=cols)
shakespeare_wiki_df = pd.DataFrame(index=sentences, columns=cols)
wiki_shakespeare_df = pd.DataFrame(index=sentences, columns=cols)

norm_df = pd.DataFrame(index=sentences, columns=norm_columns)
for sentence in sentences:
    norm_df.loc[sentence]['web'] = utils.generate(web_causal_model, sentence, gpt_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
    norm_df.loc[sentence]['wiki'] = utils.generate(wiki_causal_model, sentence, gpt_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
    norm_df.loc[sentence]['shakespeare'] = utils.generate(shakespeare_causal_model, sentence, gpt_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
norm_df.to_csv(os.path.join(CSV_DIR, 'generation', 'norm.csv'))

for layers in swap_columns:
    for layer in layers:
        layer = layer.__repr__()        
        web_wiki_model = utils.frankenstein(web_causal_model, wiki_causal_model, layer)
        web_shakespeare_model = utils.frankenstein(web_causal_model, shakespeare_causal_model, layer)
        shakespeare_wiki_model = utils.frankenstein(shakespeare_causal_model, wiki_causal_model, layer)
        wiki_shakespeare_model = utils.frankenstein(wiki_causal_model, shakespeare_causal_model, layer)
        for sentence in sentences:
            web_wiki_df.loc[sentence][layer] = utils.generate(web_wiki_model, sentence, gpt_tokenizer)
            web_shakespeare_df.loc[sentence][layer] = utils.generate(web_shakespeare_model, sentence, gpt_tokenizer)
            shakespeare_wiki_df.loc[sentence][layer] = utils.generate(shakespeare_wiki_model, sentence, gpt_tokenizer)
            wiki_shakespeare_df.loc[sentence][layer] = utils.generate(wiki_shakespeare_model, sentence, gpt_tokenizer)
web_wiki_df.to_csv(os.path.join(CSV_DIR, 'generation', 'web_wiki.csv'))
web_shakespeare_df.to_csv(os.path.join(CSV_DIR, 'generation', 'web_shakespeare.csv'))
shakespeare_wiki_df.to_csv(os.path.join(CSV_DIR, 'generation', 'shakespeare_wiki.csv'))
wiki_shakespeare_df.to_csv(os.path.join(CSV_DIR, 'generation', 'wiki_shakespeare.csv'))

In [54]:
print('===== norm df =====')
display(norm_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== web-wiki df =====')
display(web_wiki_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== web-shakespeare df =====')
display(web_shakespeare_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== shakespeare-wiki df =====')
display(shakespeare_wiki_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== wiki-shakespeare df =====')
display(wiki_shakespeare_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))

===== norm df =====


,web,wiki,shakespeare
A,"A 1.8 mm x 10.7 mm in diameter), a significant improvement over the last 2 weeks (fig. S2B, D), which also showed no improvements on the BNP1-γ- and catecholamines- and the AFR-like activity at 24°C (fig. S2B, D). The reduction of the sensitivity and specificity of this signaling gene in humans has been confirmed by a number of animal models, including the B-glu assay, as well as by the use of the fluorescence/gamma assay, which mimicked the signal induced by the E-kA1 signal. However, despite the improvement in the M-C isoforms exhibited, the reduction of this gene in the expression of this gene in humans has not been noted. Vitamins A 2.2, D 2.5, D 2.9, and MAA A receptor activity in the CNS also showed significant reductions in the C 1, D 2.9- and AFR-like activity. A large reduction in T3-A1-γ-activated C 5‐ and C 2‐betflaccidiol/flaccidiol/flaccidiol/flacc","A : To make the game a "" fun and meaningful experience "", David Gainsbourg of GameSpot says : "" It is a well @-@ made experience, and we are doing it with a lot of dedication and the ability to put the community out there so that no matter how hard we work we can make something good. It is an amazing experience, and we think we can do better with that experience. "" = = Early development = = During development of the game, David Gainsbourg worked as co @-@ designer with Chris Anderson, who had previously worked on a new, pre @-@ developed game called Starfall, which was due in January 2013. The game was slated to be released as the PlayStation 3 at the end of 2014, but Gainsbourg decided to wait for development. The game was then announced, with only the first few days of release date being considered, and Gainsbourg decided to re @-@ direct the development of the game at the end of 2013. Although Chris's former role was a precursor to Starfall, the game was first published with the game having been conceived as a standalone and unfinished game. The development team was led by Anderson","A You shall live the day in that place that we shall be spurious. Nurse: Come, marry me not to your hand. COPYRIGHT: My lord, if I had to be as virtuous as this, then, would I not have done so myself, with such honesty and modesty as this. COPYRIGHT: I, my lord, you may, of course. Nurse: But, God, if I had to please you, I would not. COPYRIGHT: Hast thou hadst done so? for if I had to come within my right, I would have done so myself, in a strange way. Nurse: The first, by any means possible, is that by any means possible, that you have done so, and therefore you have come. Nurse: For to be able to play it, so by any means possible, would I not have done so myself. COPYRIGHT: I had, my lord. Nurse: Because I am good at it, that you may have done so"
As of,"As of the last 10 days, nearly 2,000 people have died from liver cancer. The Centers for Disease Control and Prevention reports that more than 2 million Americans are dying of liver cancer each year. Over the past decade, there have been roughly 1.7 million deaths from coronary heart disease. In 2007, the U.S. Department of Health and Human Services approved a number of new liver biostics that have been approved for the diagnosis. The FDA announced in February that it had approved two types of liver biostics called “Gluenza Biogenesis” in 2007, which contains antibodies to influenza. In 2007, the FDA approved the first category of clinical liver biostics for treating liver problems using Hepatitis A, and it’s been approved for routine testing of liver biochemistry by the FDA as part of its long-term management. A similar biopsy for liver disease is in its final stages. Copyright 2015 KING.COM Click here to read more about the latest developments in liver biogenesis.NEW YORK (KDKA) -- The New York Times on Sunday reported on the devastating news that a single, high-profile victim, with his wife and five","As of 1995, the International Film Classification Bureau ( IDB ) designated the film as an under @-@ 

===== web-wiki df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
A,"A by George W. Bush. ""No, this is not a presidential race,"" he wrote. ""It is a campaign that makes the United States the most democratic nation. It is a campaign where we all have a responsibility to have the right to say no."" Advertisement Continue reading the main story The campaign also claims to have produced a videotape of Donald Trump talking about an ""Unilateral Iraq War"" that was released earlier this month. In a statement to reporters on Tuesday, Mr. Bush defended the video, saying he ""tried to help bring the tapes to our attention while traveling for a campaign event in the United States."" Newsletter Sign Up Continue reading the main story Please verify you're not a robot by clicking the box. Invalid email address. Please re-enter. You must select a newsletter to subscribe to. Sign Up You will receive emails containing news content, updates and promotions from The New York Times. You may opt-out at any time. You agree to receive occasional updates and special offers for The New York Times's products and services. Thank you for subscribing. An error has occurred. Please try again later. View all New York Times newsletters. On Wednesday, Mr. Obama","A, VOTRO, TAKON, VAWD, VERNARD, VERTANA, VERTANA, VERDE, VERIN, VERDE I’m sure, some will like this! You see, all the money has gone down, all the expenses have gone down! The United States has yet another massive war in Afghanistan. It’s been more than a decade since the Great War and it’s been on a remarkable scale. On the first day in June 1965, President Lyndon B. Johnson announced his withdrawal from the war, which was going to be called Operation Desert Storm. At 5:40 a.m. ET, in his first post-Overnight announcement, Secretary of Defense Robert Gates stated in an interview that he was withdrawing the war if he could. President Johnson, who was a member of the State Department on the ground, insisted he could not participate in the bombing at that time and that he had not even been able to see what had happened to the troops from the ground. At the same time, he said he would do his utmost to help protect their families in the event of an attack on their house.","A. A. A. B., G. Fuchs, S. E.M. et al., “Aquino B cells show a novel and novel form of polymerase activity,” The Journal of Cell Metabolism. 9 ( 2011 ), DOI 6.6-722-1. ( doi:10.1038/j.11044-2011.14) doi:10.1038/j.11245.1112) 10.1016/j.cmatagl.2011.03.004) ) 22.5% For the sake of brevity I’m here to explain some of the most important variables in the B cell. B cell viability has been shown to be a fundamental issue with many B cells (particularly B1 and B2) with an increase in their length. However, many of the cells are made up of only around a couple of immature and highly active, as well as many of the immature cells that were taken from them and subjected to the addition of lectin. As with all of the cell types B cell types SIT-1–3–3, these types of cells are in all cases limited to one or","A *This post was written by Steve Nadel on January 10th, 2008. My first response to the blog article was as follows: I will add that the fact that it is not an actual Google search results, or even the Google search results, does not mean that Google does not have a real result; the only thing I know about Google's results is that Google is also not the dominant search engine, so it is not a Google search result. According to Google, Google has “a history of losing relevance” to news organizations and publishers with similar problems. Indeed, such as the Huffington Post, the Guardian, and The Guardian all said their failures were due to Google's dominance of American newsrooms in terms of their coverage of news; however, they can not say definitively which of their top media sources were involved in the stories published. Even m

===== web-shakespeare df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
A,"A.com)” (This post originally appeared on this blog.) On September 10, 2007, the U.S. Justice Department filed the following brief, claiming that Cesar Chavez’s legal argument against his use of the term “political speech” was “founded to justify the practice of using words of political expression.” “If we continue to reject the usage of political speech as political speech, the language of speech we recognize as political is rooted in a political convention.” The brief was filed in response to a complaint filed by the Department of Justice in August 2007 about Cesar Chavez’s use of the term “political speech.” From that same day, President Bush took his first public appearances with Cesar Chavez in his first appearance with Cesar Chavez in Bush City. But in his third appearance in 2006, he was greeted by a host of supporters in the capital and then joined forces with Senator Richard M. Murphy to demand that he refrain from using his speech. The issue was settled after the court was assigned to the Supreme Court. President Bush responded in his second appearance with a brief expressing his opposition to Cesar Chavez in September 2007 and reaffirmed the right of","A the most extreme and perhaps most fundamental elements to the “natural system”, the fact that the government of the United States has been subject to a systematic assault by foreign and domestic officials. This, of course, is, again, due to the fact that the Federal Government has continued to function under the direction of a president. Of course, the point here is that the United States can’t be trusted to serve in this situation because foreign powers are not the primary actors in the matter. This is a point that should be taken seriously, and especially because the Constitution explicitly prohibits foreign powers from acting unilaterally and in its entirety. Under the United States Constitution, the government can be trusted to make necessary decisions which “beyond the power delegated to Congress.” And that is a fundamental issue for the United States of America if it is to remain a “one nation”, because a national executive does not require members of the federal government to act on the behalf of a foreign power. Yet the most fundamental elements of this dispute are also under dispute—the government can act on behalf of the United States. If the foreign powers are to act in this manner, it is not that they are not","Aaike, and for every human on our planet we are all a part of—there is not one.” He said a smile. “Yes, we are. And no one can change it,” he paused. Advertisement Continue reading the main story Mr. Trump’s win in the Senate is not just a testament to how deeply Mr. Romney was in office. It is an image that he could be trusted. Indeed, it was a triumph in the face of Republicans at the polls. Photo “This is so deeply personal. It’s really a tragedy,” Mr. Trump said, “for the Republican Party, and for our country,” a day after the election as the Republican convention. ""There’s going to be problems in the Republican Party. It will be the party where you have a winning majority, you have a president who wants to be able to defeat you.” “But it’s a mistake. I, on the other hand,” Mr. Trump told a packed room at a campaign event this week, “for it to be a disaster for my country, I","A - With the new U.S. embassy building, there is no indication of the presence of an African American person. But in a telephone interview that aired on Thursday, Trump said there were no African Americans at the building in question. ""It's very, very hard to be sure. There is something there that nobody said and they are going to the wrong place, and I think people say, OK, maybe no African American in fact has a good way to be here,"" he said. There is no evidence of racism in the Un

===== shakespeare-wiki df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
A,"A Wexford Second of July: The Duke of Hereford has taken his court, and he's been put to trial; the Duke is now given a pardon; and Weeping, weeping, weeps and clamors, the Duke has gone from trial, The new king of England has sworn. Second of July: He will be our king's father. PERSUCE: Sir, let us here: He hath had no more power then Henry; and yet, a greater power Of this royal power, he hath sworn. DUKE VINCENTIO: I will speak for a hundred or so thousand words: let's be with you. DUKE VINCENTIO: Sir, that we have all made so much good of him; let's say we hear. MENENIUS: Great good Henry; for the good of you? LUCIO: A thousand words: A thousand words: That is one word of many. MENENIUS: Nay, the word is one word of many, with this one word to say: The","A A noble lady and The daughter of two noblemen; A young man who loves her; A woman of five years' worth; A boy named Isabel, a good-natured child, And the youngest of seven. MENENIUS: The daughter of seven-and-a-half years' worth; A good-natured boy: it is not a very high worth; Which I have to pay him to, Which you know will bear to keep him; And I do promise him a good life, That his father will, I have to pay me some of his money. LADY MARGARET: He will come; my name will stand, And give him some money, And his heart will beat! MENENIUS: The daughter of three and a half years' worth; And he shall bear him back: Which you know he will bear to keep him, Which is the word I will bear to stay with. LADY MARGARET: And my name will be a great man; And let him not be, my brother.","A of us, for I do believe God is with me, for I will make no further use of any less that I have power. Heaven forbid, if any one so say the church: The church you call a church is to do so Which is my duty. But with the power of God, for once in heaven, That no power in heaven will but lead to the destruction of any other Like a thing hath power; That all things which I have power cannot not lead To ruin this world. I am not a man: Yet if one man can do the same thing, That one woman knows a man can not do the same thing. But what then is the reason? QUEEN MARGARET: This is the reason I have to tell you how I know a man. QUEEN MARGARET: A man's heart should not be at war; he is not a man, I am a God, but a God; God's power, in my judgment, may be my Committee. Therefore, this is my duty. QUEEN MARGARET: A man's","A in the midst of grief. He had gone from my house and got here and there, but had gone to his house, but had gone In search of his wife for my husband's aid. FUTURE: Come, he dined, I know you well. SAMUEL GAUNT: It looks in the shadows, with an eye and face, Shall come here and seek Your advice. COMINIUS: I am your counsel To send you hither for me, and bring me Your counsel at your pleasure! SAMUEL GAUNT: I know well, I know better. CAMILLO: Where have you come? SAMUEL GAUNT: I know you well. COMINIUS: Well, sir, where, but when, he found you and the lady As we were, he found your aid, To be fair, I am a prisoner, sir, He took you to his house, where we were, It was not too late, but did not end us. I had known him well that you, my","A, thou hast done what we said; I will not, to be silent: when I was the prisoner of the prisoner, I was very silent. BUSH: I'll leave it to you, my servant; I'll swear not. KING RICHARD III: For thou shalt not be silent, thou shalt hear me answer. LUCIO: Away to thy brother, and to thy friends. LUCIO: I am not dead, my lord, now And my soul's blood's blood's blood's blood's blood's blood's blood's blood's blood's blood's blood's blood's You shall see, and how is it ever so dull, and how how long? BUSH: I say to my son, farewell, and a welcome to your bosom, From my dear daughter's bosom; To our son, his hand is too long, and his heart, To let me pass in and forth to that

===== wiki-shakespeare df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
A,"A to provide him with the necessary facilities. The last of these was the railway at Shola, a port for convoys from the Indian war. The port is situated opposite the main highway of Portugliano, and is a port for the first phase of operations. In April, the Royal Navy's first operation on Shola reached the port of Portugliano, with an increase in personnel. In March, the Navy's second operation against the Ottoman Empire was undertaken by Prince Viscount of Bathurst. In April, Admiral Krantik served as the bridgehead of the Ottoman Empire. There, Admiral Krantik served as the vessel's first operational officer. A naval exercise conducted with the assistance of the German and Ottoman naval forces was concluded between June and September, and the two ships operated by the German and Ottoman armies continued north on the Mediterranean Sea to the east. A French naval exercise was conducted between November and December in the north of the channel. A German and Ottoman naval action at Portugliano, on the left and the right, consisted of an engagement with the Ottoman naval forces on the west coast of Portugliano, on the left and the right, which both sides of","A apron of a man that was raised to the highest rank on the force, in a way that is neither unusual nor uncommon. He was raised by the Order of the Order of the Order of the Order of the Order of the Templars and of the Order of the Templars, which was under the command of King Henry VIII, whose son Edward IV was raised to the ranks of the Order of the Templars by Edward II, who was raised by the Order of the Order of the Templars by the Order of the Templars. He was not the only one who was raised to the highest rank on the force, and the only one whose sons Henry VIII was raised to the rank of the Order of the Templars. The only other of the Order of the Templars, George VI, was raised by the Order of the Order of the Templars and his brother George III, who was raised by his father. The other, James II, is raised by the Order of the Templars and his brother George III, who was raised by the Order of the Templars. The other was raised by the Order of the Templars and his brother George III. The other was raised by the Order of the Templars and his brother George","A : I must say that it is not quite enough, that they would think,'As a man who's not too far from that sort of person's personality is the same and yet a great deal of respect is paid in the same way as what's done to him, when's that right?' "" An interesting analysis of the relationship between the three roles on both sets of novels has come to the attention of The Sunflower Project The Sunflower Project was published in November 1941. It was based on a review of The Sunflower Project by The Guardian Newspaper ; in order to give it a wider international audience, it was published by the London based newspaper The Guardian and distributed by the Royal Canadian Press in France. It is the second novel for the series and was based on the same work. On July 1, 1941, following news of the death of Mr. Burns's son, John, it was published by B.C. Paper and in the New Statesman ; in May 1941, by The Guardian Newspapers it was broadcast by the Royal Canadian Press and printed by The Guardian Newspapers in England. John's death was condemned in the British media, and on 9 August he was","A @-@ game, on the way to a concert hall. When the crowd was large enough to see the game, and the arena was empty for a concert, they decided to hold a concert outside the venue. This included the first leg of a concert in the United States held in 2005, and a second leg of a concert in the United Kingdom on November 15, 2006. = = = Guitar Hero 1 = = = = The Guitar Hero series, which was first released in 1995 with the Guitar Hero game, is an entirely n

### MLM models

In [46]:
roberta_tokenizer = AutoTokenizer.from_pretrained('distilroberta-base', use_fast=True)
wiki_mlm_model = torch.load(os.path.join(MLM_DIR, 'distilroberta-base_wikitext_2109080109.pkl'))
shakespeare_mlm_model = torch.load(os.path.join(MLM_DIR, 'distilroberta-base_tiny_shakespeare_2109080209.pkl')) # distilroberta-base_tiny_shakespeare_2109071609.pkl
web_mlm_model = torch.load('/content/drive/MyDrive/NLP_project/trained_models/mlm/distilroberta-base_stas/openwebtext-10k_2109081809.pkl')

In [61]:
M = roberta_tokenizer.mask_token
sentences = [f'Make {M}, there is no time',
             f'This {M} is very tasty',
             f'This fruit is very {M}',
             f'I wish I were a {M}',
             f'I wish I {M} a fish',
             f'I wish I were {M} fish',
             f'The rain in {M} falls mainly on the plain',
             f'The rain in spain falls mainly on the {M}',
             f'The rain in spain falls mainly on {M} plain',
             f'The rain in spain falls mainly {M} the plain',
             f'The rain {M} spain falls mainly on the plain']
norm_columns = ['web', 'wiki', 'shakespeare']
swap_columns = [[[i] for i in range(6)],
                [[i, i+1] for i in range(5)],
                [[i, i+1, i+2] for i in range(4)],
                [[i, i+1, i+2, i+3] for i in range(3)],
                [[0,1,2,3,4,5]]]
cols = []
for columns in swap_columns:
    for layer in columns:
        cols.append(layer.__repr__())
web_wiki_df = pd.DataFrame(index=sentences, columns=cols)
shakespeare_wiki_df = pd.DataFrame(index=sentences, columns=cols)
web_shakespeare_df = pd.DataFrame(index=sentences, columns=cols)

norm_df = pd.DataFrame(index=sentences, columns=norm_columns)
for sentence in sentences:
    norm_df.loc[sentence]['web'] = utils.mlm(web_mlm_model, sentence, roberta_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
    norm_df.loc[sentence]['wiki'] = utils.mlm(wiki_mlm_model, sentence, roberta_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
    norm_df.loc[sentence]['shakespeare'] = utils.mlm(shakespeare_mlm_model, sentence, roberta_tokenizer).replace('\t\n\t\n', '\t\\n\t\n')
norm_df.to_csv(os.path.join(CSV_DIR, 'mlm', 'norm.csv'))

for layers in swap_columns:
    for layer in layers:
        layer = layer.__repr__()        
        web_wiki_model = utils.frankenstein(web_mlm_model, wiki_mlm_model, layer)
        web_shakespeare_model = utils.frankenstein(web_mlm_model, shakespeare_mlm_model, layer)
        shakespeare_wiki_model = utils.frankenstein(shakespeare_mlm_model, wiki_mlm_model, layer)
        for sentence in sentences:
            web_wiki_df.loc[sentence][layer] = utils.mlm(web_wiki_model, sentence, roberta_tokenizer)
            web_shakespeare_df.loc[sentence][layer] = utils.mlm(web_shakespeare_model, sentence, roberta_tokenizer)
            shakespeare_wiki_df.loc[sentence][layer] = utils.mlm(shakespeare_wiki_model, sentence, roberta_tokenizer)
web_wiki_df.to_csv(os.path.join(CSV_DIR, 'mlm', 'web_wiki.csv'))
web_shakespeare_df.to_csv(os.path.join(CSV_DIR, 'mlm', 'web_shakespeare.csv'))
shakespeare_wiki_df.to_csv(os.path.join(CSV_DIR, 'mlm', 'shakespeare_wiki.csv'))

In [62]:
print('===== norm df =====')
display(norm_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== web-wiki df =====')
display(web_wiki_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== web-shakespeare df =====')
display(web_shakespeare_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))
print('===== shakespeare-wiki df =====')
display(shakespeare_wiki_df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap', 'line-height': '1.25'}))

===== norm df =====


,web,wiki,shakespeare
"Make , there is no time",0.284: it 0.28: sure 0.0407: up 0.0399: sense 0.0349: mistakes,0.584: sure 0.148: it 0.05: up 0.0337: clear 0.016: no,0.264: haste 0.174: sure 0.165: it 0.0314: good 0.0309: now
This is very tasty,0.156: recipe 0.145: one 0.143: dish 0.0465: bread 0.0399: sauce,0.159: dish 0.132: one 0.0714: recipe 0.0652: sauce 0.038: bread,0.22: dish 0.107: one 0.0557: bread 0.0541: meal 0.0428: recipe
This fruit is very,0.15: tasty 0.112: good 0.0916: special 0.0814: delicious 0.0528: sweet,0.0924: tasty 0.0906: delicious 0.0632: rare 0.0602: special 0.0505: expensive,0.267: ripe 0.155: sweet 0.0579: tasty 0.0554: delicious 0.0347: bitter
I wish I were a,0.0791: robot 0.0662: doctor 0.0638: hero 0.0568: fan 0.0491: man,"0.144: 0.123: "" 0.0426: fan 0.0401: . 0.0396: man",0.0938: man 0.0713: king 0.0588: god 0.0497: boy 0.0423: prince
I wish I a fish,0.809: had 0.0733: were 0.046: was 0.00989: wanted 0.00892: saw,0.629: had 0.196: was 0.13: were 0.00808: wanted 0.00512: knew,0.498: had 0.368: were 0.0585: was 0.0204: 'd 0.0115: saw
I wish I were fish,0.652: a 0.0994: to 0.029: more 0.0275: bigger 0.0254: the,0.204: alive 0.0927: . 0.0819: better 0.0761: to 0.0625: a,0.703: a 0.0721: to 0.0305: the 0.0283: more 0.0261: bigger
The rain in falls mainly on the plain,0.134: London 0.0669: Britain 0.0603: Bangladesh 0.0478: Barcelona 0.0395: Spain,0.153: Britain 0.118: London 0.0631: Kent 0.0425: August 0.0408: England,0.149: London 0.0545: summer 0.0542: Britain 0.046: England 0.0441: winter
The rain in spain falls mainly on the,0.0894: trees 0.0857: farmers 0.0853: poor 0.0429: wind 0.036: farmer,0.308: . 0.109: poor 0.0565: roofs 0.0531: roof 0.0468: trees,0.108: ground 0.106: trees 0.0889: wind 0.052: roof 0.0442: sky
The rain in spain falls mainly on plain,0.679: the 0.0654: a 0.0429: flood 0.031: coastal 0.0206: salt,0.418: the 0.0514: a 0.0403: agriculture 0.0379: farmland 0.0355: land,0.892: the 0.0578: a 0.0198: this 0.00573: flood 0.00389: sea
The rain in spain falls mainly the plain,0.451: on 0.206: in 0.187: from 0.0454: over 0.0283: to,0.372: on 0.233: in 0.143: from 0.0764: over 0.0398: along,0.322: on 0.285: in 0.153: from 0.1: to 0.0285: over


===== web-wiki df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
"Make , there is no time",0.499: sure 0.253: it 0.0311: sense 0.0217: haste 0.0194: up,0.363: sure 0.308: it 0.0423: up 0.0379: sense 0.0275: haste,0.335: it 0.296: sure 0.0396: haste 0.0378: up 0.0375: sense,0.311: it 0.303: sure 0.0366: mistakes 0.0352: up 0.0316: sense,0.289: it 0.274: sure 0.0541: up 0.033: money 0.0319: haste,0.358: sure 0.233: it 0.0377: up 0.0358: money 0.0326: now,0.546: sure 0.246: it 0.0261: sense 0.0247: up 0.0176: haste,0.355: sure 0.322: it 0.0464: up 0.0395: sense 0.0315: haste,0.382: it 0.27: sure 0.0356: haste 0.0348: up 0.0304: sense,0.328: it 0.298: sure 0.0454: up 0.032: haste 0.0312: friends,0.357: sure 0.234: it 0.0502: up 0.0366: money 0.0329: now,0.513: sure 0.251: it 0.0302: sense 0.0268: up 0.0201: clear,0.368: it 0.314: sure 0.0449: up 0.0352: sense 0.0263: haste,0.386: it 0.252: sure 0.0528: up 0.0465: haste 0.0309: friends,0.41: sure 0.239: it 0.0395: up 0.0334: now 0.0331: money,0.472: sure 0.292: it 0.0275: sense 0.0269: up 0.023: clear,0.356: it 0.286: sure 0.0762: up 0.0338: haste 0.0241: sense,0.353: sure 0.282: it 0.0461: up 0.0436: haste 0.0283: money,0.56: sure 0.175: it 0.0397: up 0.0293: clear 0.0162: friends
This is very tasty,0.135: dish 0.123: one 0.104: recipe 0.0555: sauce 0.0519: bread,0.158: dish 0.144: one 0.0886: recipe 0.0503: bread 0.0497: sauce,0.191: recipe 0.149: dish 0.0917: one 0.0474: bread 0.0462: sauce,0.204: one 0.159: recipe 0.153: dish 0.0445: sauce 0.0346: bread,0.181: one 0.135: recipe 0.111: dish 0.0463: sauce 0.0401: bread,0.228: recipe 0.166: one 0.0938: dish 0.0412: bread 0.0325: stuff,0.143: dish 0.131: one 0.0678: recipe 0.0631: sauce 0.0551: bread,0.174: dish 0.102: recipe 0.0963: one 0.0565: sauce 0.0532: bread,0.18: recipe 0.157: dish 0.129: one 0.0541: sauce 0.0376: beer,0.229: one 0.132: recipe 0.119: dish 0.0531: sauce 0.0431: beer,0.21: one 0.207: recipe 0.0709: dish 0.0357: bread 0.0315: beer,0.167: dish 0.0875: one 0.083: recipe 0.0685: sauce 0.0556: bread,0.179: dish 0.143: one 0.0942: recipe 0.0598: sauce 0.0397: bread,0.155: one 0.142: recipe 0.122: dish 0.0747: sauce 0.0421: beer,0.257: one 0.208: recipe 0.076: dish 0.0336: beer 0.0314: sauce,0.173: dish 0.133: one 0.0759: recipe 0.0668: sauce 0.0511: beer,0.169: one 0.13: dish 0.0825: sauce 0.0738: recipe 0.0397: beer,0.233: recipe 0.192: one 0.0779: dish 0.0459: sauce 0.0312: beer,0.238: one 0.109: recipe 0.0839: dish 0.0619: sauce 0.0415: beer
This fruit is very,0.12: special 0.115: tasty 0.0839: good 0.077: delicious 0.0522: sweet,0.148: tasty 0.0959: good 0.0885: delicious 0.0734: special 0.057: sweet,0.142: tasty 0.101: good 0.0817: delicious 0.0804: special 0.0547: sweet,0.151: tasty 0.112: good 0.0929: special 0.0908: delicious 0.0693: sweet,0.144: tasty 0.108: good 0.0833: special 0.0751: delicious 0.0584: sweet,0.149: tasty 0.103: good 0.0713: delicious 0.0663: special 0.0544: sweet,0.103: tasty 0.0977: special 0.08: delicious 0.0699: good 0.0606: sweet,0.134: tasty 0.0868: good 0.0859: delicious 0.0694: special 0.0538: sweet,0.141: tasty 0.0969: good 0.0876: delicious 0.0837: special 0.0707: sweet,0.147: tasty 0.106: good 0.0867: delicious 0.0855: special 0.079: sweet,0.144: tasty 0.103: good 0.0659: delicious 0.0589: sweet 0.0575: special,0.0892: special 0.0821: tasty 0.0706: delicious 0.0671: good 0.0527: sweet,0.136: tasty 0.0914: delicious 0.0803: good 0.0757: special 0.0669: sweet,0.126: tasty 0.0937: good 0.083: sweet 0.0801: delicious 0.0754: special,0.151: tasty 0.102: good 0.0844: sweet 0.0769: delicious 0.0579: special,0.104: special 0.0824: tasty 0.0705: delicious 0.0621: sweet 0.0579: good,0.124: tasty 0.0841: delicious 0.0801: good 0.0777: sweet 0.0725: special,0.126: tasty 0.0917: sweet 0.0875: good 0.0713: delicious 0.0528: special,0.0737: tasty 0.0698: special 0.0686: sweet

===== web-shakespeare df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
"Make , there is no time",0.454: sure 0.28: it 0.0357: haste 0.0252: sense 0.0197: up,0.369: sure 0.29: it 0.0401: haste 0.0354: up 0.0317: sense,0.346: it 0.253: sure 0.0682: haste 0.0411: sense 0.0321: up,0.326: it 0.213: sure 0.0406: money 0.0373: now 0.0369: haste,0.241: it 0.222: sure 0.0526: time 0.0474: haste 0.0382: up,0.244: it 0.212: sure 0.0447: up 0.0404: now 0.0365: time,0.486: sure 0.272: it 0.0497: haste 0.0237: up 0.0201: sense,0.336: sure 0.327: it 0.0785: haste 0.0321: up 0.032: sense,0.369: it 0.206: sure 0.0857: haste 0.0398: now 0.032: sense,0.26: it 0.154: sure 0.0751: time 0.0699: haste 0.0489: money,0.211: it 0.148: sure 0.0795: time 0.0414: now 0.0404: up,0.441: sure 0.3: it 0.0764: haste 0.0214: sense 0.0209: up,0.343: it 0.288: sure 0.109: haste 0.029: up 0.0252: sense,0.288: it 0.132: haste 0.125: sure 0.0576: time 0.0493: now,0.224: it 0.106: sure 0.105: time 0.0541: now 0.0491: haste,0.377: sure 0.33: it 0.0993: haste 0.0234: clear 0.0196: up,0.279: it 0.178: sure 0.175: haste 0.0396: up 0.0322: now,0.239: it 0.0895: time 0.0864: haste 0.0794: sure 0.0716: now,0.25: it 0.167: sure 0.133: haste 0.0427: clear 0.034: up
This is very tasty,0.165: dish 0.106: one 0.0904: recipe 0.0605: bread 0.053: sauce,0.141: one 0.14: dish 0.101: recipe 0.0467: bread 0.0383: sauce,0.231: recipe 0.128: dish 0.117: one 0.0381: beer 0.0357: bread,0.181: one 0.174: recipe 0.169: dish 0.0417: bread 0.0363: sauce,0.153: recipe 0.14: dish 0.132: one 0.0451: bread 0.0361: beer,0.201: one 0.154: recipe 0.0895: dish 0.0487: bread 0.035: stuff,0.145: dish 0.125: one 0.0651: recipe 0.0618: bread 0.0487: sauce,0.141: recipe 0.132: dish 0.125: one 0.0368: beer 0.0354: bread,0.237: recipe 0.142: dish 0.136: one 0.0392: beer 0.0313: bread,0.174: dish 0.151: one 0.15: recipe 0.044: bread 0.0347: beer,0.201: one 0.145: recipe 0.081: dish 0.0444: bread 0.0345: food,0.153: dish 0.104: one 0.096: recipe 0.0456: bread 0.0401: sauce,0.152: one 0.146: dish 0.132: recipe 0.0386: beer 0.0346: soup,0.195: recipe 0.148: dish 0.127: one 0.0383: beer 0.0367: bread,0.234: one 0.147: recipe 0.0969: dish 0.0431: bread 0.043: food,0.168: dish 0.138: one 0.0872: recipe 0.0444: beer 0.0428: bread,0.141: dish 0.137: one 0.107: recipe 0.0399: bread 0.0384: beer,0.204: one 0.186: recipe 0.0837: dish 0.0441: food 0.0365: bread,0.212: one 0.085: dish 0.0765: recipe 0.0492: bread 0.0395: food
This fruit is very,0.137: tasty 0.0945: delicious 0.0926: special 0.0731: sweet 0.0717: good,0.151: tasty 0.0981: good 0.0765: delicious 0.0674: special 0.0534: sweet,0.151: tasty 0.0984: good 0.0777: delicious 0.0729: special 0.0488: sweet,0.16: tasty 0.121: good 0.0881: delicious 0.0752: sweet 0.0694: special,0.126: tasty 0.106: good 0.0932: special 0.0752: sweet 0.0697: delicious,0.129: tasty 0.0921: good 0.0665: special 0.0662: delicious 0.0634: sweet,0.132: tasty 0.0874: delicious 0.0757: sweet 0.0735: special 0.0671: good,0.151: tasty 0.0862: good 0.069: delicious 0.0576: special 0.048: important,0.166: tasty 0.108: good 0.0853: delicious 0.0599: sweet 0.0552: special,0.125: tasty 0.113: good 0.104: sweet 0.0722: delicious 0.0715: special,0.105: tasty 0.0889: good 0.0827: sweet 0.0603: special 0.0539: delicious,0.136: tasty 0.0812: delicious 0.0686: special 0.0665: good 0.058: sweet,0.165: tasty 0.0923: good 0.0783: delicious 0.0483: sweet 0.0467: special,0.119: tasty 0.102: good 0.0867: sweet 0.077: ripe 0.0702: delicious,0.11: sweet 0.0986: tasty 0.0958: good 0.0872: ripe 0.0523: delicious,0.142: tasty 0.0886: delicious 0.0701: good 0.0684: sweet 0.0579: ripe,0.118: tasty 0.0899: ripe 0.0873: good 0.0709: sweet 0.0662: delicious,0.101: ripe 0.0912: sweet 0.0869: good 0.0858: tasty 0.048: delicious,0.142: ripe 0.0949: sweet 0.0663: tasty 0.0608: good 0.045: delicious
I wish I were 

===== shakespeare-wiki df =====


,[0],[1],[2],[3],[4],[5],"[0, 1]","[1, 2]","[2, 3]","[3, 4]","[4, 5]","[0, 1, 2]","[1, 2, 3]","[2, 3, 4]","[3, 4, 5]","[0, 1, 2, 3]","[1, 2, 3, 4]","[2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
"Make , there is no time",0.214: sure 0.198: haste 0.155: it 0.0371: now 0.0368: no,0.19: haste 0.178: sure 0.167: it 0.0375: now 0.034: no,0.225: sure 0.188: haste 0.17: it 0.0367: good 0.0361: clear,0.243: sure 0.214: haste 0.174: it 0.0333: no 0.0278: up,0.274: sure 0.249: haste 0.172: it 0.0326: clear 0.0296: up,0.352: sure 0.265: haste 0.13: it 0.0255: now 0.0234: good,0.211: sure 0.153: it 0.128: haste 0.0438: no 0.0433: now,0.242: sure 0.161: it 0.12: haste 0.0425: clear 0.0378: no,0.299: sure 0.18: it 0.139: haste 0.0371: clear 0.0335: no,0.37: sure 0.177: haste 0.17: it 0.0333: clear 0.0311: up,0.468: sure 0.212: haste 0.129: it 0.0218: up 0.0207: now,0.281: sure 0.147: it 0.0795: haste 0.0477: no 0.0342: clear,0.315: sure 0.165: it 0.0874: haste 0.0437: clear 0.0419: no,0.425: sure 0.174: it 0.104: haste 0.0459: clear 0.0374: up,0.565: sure 0.136: haste 0.126: it 0.0226: up 0.0185: clear,0.36: sure 0.142: it 0.0575: haste 0.0531: no 0.0345: up,0.441: sure 0.156: it 0.0638: haste 0.0544: clear 0.0411: up,0.611: sure 0.131: it 0.0772: haste 0.0276: up 0.0242: clear,0.682: sure 0.0993: it 0.0283: haste 0.0248: up 0.0208: clear
This is very tasty,0.202: dish 0.108: one 0.0543: bread 0.0483: meal 0.0462: recipe,0.241: dish 0.112: one 0.0606: bread 0.0421: meal 0.0375: fruit,0.214: dish 0.102: one 0.0679: bread 0.0448: meal 0.0384: recipe,0.214: dish 0.114: one 0.0496: recipe 0.0465: meal 0.0456: bread,0.2: dish 0.13: one 0.0491: bread 0.0447: meal 0.0435: recipe,0.225: dish 0.0836: one 0.0732: recipe 0.0573: meal 0.0489: bread,0.234: dish 0.109: one 0.0582: bread 0.0408: recipe 0.0374: meal,0.234: dish 0.102: one 0.0729: bread 0.0381: fruit 0.0357: meal,0.211: dish 0.107: one 0.0551: bread 0.0453: recipe 0.0383: meal,0.19: dish 0.141: one 0.0507: recipe 0.0389: bread 0.0387: meal,0.199: dish 0.0998: one 0.0731: recipe 0.0482: meal 0.0429: bread,0.233: dish 0.0985: one 0.0694: bread 0.0383: fruit 0.0369: recipe,0.234: dish 0.108: one 0.0583: bread 0.0395: sauce 0.0394: recipe,0.183: dish 0.134: one 0.0487: sauce 0.0475: bread 0.0462: recipe,0.186: dish 0.111: one 0.0831: recipe 0.0416: meal 0.0368: sauce,0.228: dish 0.0996: one 0.0564: bread 0.045: recipe 0.0448: sauce,0.204: dish 0.134: one 0.0584: sauce 0.0492: bread 0.0406: recipe,0.183: dish 0.109: one 0.0746: recipe 0.0481: sauce 0.0417: bread,0.198: dish 0.102: one 0.0747: recipe 0.0676: sauce 0.0433: bread
This fruit is very,0.199: ripe 0.134: sweet 0.0577: tasty 0.0571: delicious 0.0411: rare,0.217: ripe 0.173: sweet 0.0649: delicious 0.0604: tasty 0.0314: .,0.258: ripe 0.179: sweet 0.0555: delicious 0.0527: tasty 0.0387: bitter,0.248: ripe 0.172: sweet 0.0642: tasty 0.0639: delicious 0.0375: rare,0.216: ripe 0.15: sweet 0.094: tasty 0.077: delicious 0.0293: good,0.174: sweet 0.161: ripe 0.111: tasty 0.0799: delicious 0.0356: rare,0.15: ripe 0.142: sweet 0.0636: delicious 0.0567: tasty 0.0381: rare,0.196: sweet 0.184: ripe 0.0653: delicious 0.0563: tasty 0.0355: .,0.225: ripe 0.192: sweet 0.0625: delicious 0.0583: tasty 0.0374: bitter,0.191: ripe 0.16: sweet 0.101: tasty 0.0869: delicious 0.0383: rare,0.166: tasty 0.149: sweet 0.11: ripe 0.1: delicious 0.0414: nutritious,0.15: sweet 0.124: ripe 0.0613: delicious 0.0526: tasty 0.0388: rare,0.2: sweet 0.147: ripe 0.0721: delicious 0.0627: tasty 0.0424: special,0.178: sweet 0.174: ripe 0.0883: tasty 0.0814: delicious 0.0386: rare,0.169: tasty 0.153: sweet 0.108: delicious 0.0891: ripe 0.0415: rare,0.146: sweet 0.092: ripe 0.0634: delicious 0.0586: rare 0.0547: tasty,0.182: sweet 0.106: ripe 0.0879: tasty 0.0871: delicious 0.0471: special,0.172: sweet 0.147: tasty 0.105: delicious 0.0823: ripe 0.0422: rare,0.125: sweet 0.101: tasty 0.0905: delicious 0.0725: rare 0.0704: special
I wish I were a,0.0941: man 0.0586: king 0.0553: boy 0.0415: 